# King County Homebuyer Help


## Business Understanding
- The key stakeholders for this analysis project are potential home buyers moving to the King County area
- We are using a data set of recent house sales in King County from 2021-2022 to create a predictive model
- We used predictive modeling to predict prices based on certain home features
- Our first recommendation would to be use the tool we created to get prospective homes
- Our second recommendation would be that once a potential home buyer does buy their home, to bolster certain features of their homes to add value

## Data Understanding/Data Preperation

In [1]:
# Importing all packeges required for code below
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from geopy.distance import geodesic
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Dataset
kcdf = pd.read_csv('./data/kc_house_data.csv')

## Data Cleaning

In [3]:
# Dropping date column and row with null values to clean the dataframe
kcdf = kcdf.drop(columns = 'date')
kcdf = kcdf.dropna()

- Observed not all houses in dataset were actually in King County
- Used Zip Code values to filter dataframe for only King County house sales

In [4]:
# Function to extract zipcodes from house address
def zip_code(address):
    x = address.split(' ')[-3]

    return x.split(',')[0]

In [5]:
# Creating a zip code column in the dataframe and convert to int
kcdf['zip_code'] = kcdf['address'].apply(lambda x: zip_code(x))
kcdf['zip_code'] = kcdf['zip_code'].astype(int)

In [6]:
# From government database, we inputted all zipcodes for King County and used to filter dataframe
kc_zips = [98001, 98002,98003, 98004,98005,98006, 98007,98008,98009, 98010, 98011, 98013, 98014,98015,98019,98022,98023,98024,98025,98027,98028,98029,98030,98031,98032,98033,98034,98035,98038,98039,98040,98041,98042,98045,98047,98050,98051,98052,98053,98054,98055,98056,98057,98058,98059,98062,98063,98064,98065,98070,98071,98072,98073,98074,98075,98077,98083,98089,98092,98093,98101,98102,98103,98104,98105,98106,98107,98108,98109,98111,98112,98113,98114,98115,98116,98117,98118,98119,98121,98122,98124,98125,98126,98127,98129,98131,98132,98133,98134,98136,98138,98139,98141,98144,98145,98146,98148,98151,98154,98155,98158,98160,98161,98164,98165,98166,98168,98170,98171,98174,98175,98177,98178,98181,98184,98185,98188,98190,98191,98194,98195,98198,98199,98224,98288]

realkcdf = kcdf[kcdf['zip_code'].isin(kc_zips)]

Interested in adding column for distance from Amazon HQ

In [7]:
# Zipping the columns lat/long into one column called location
realkcdf['location'] = list(zip(realkcdf.lat, realkcdf.long))

<ipython-input-7-76edfb550a8f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  realkcdf['location'] = list(zip(realkcdf.lat, realkcdf.long))


In [8]:
# Amazon headquarter coordinates
amzn_coord = (47.615722, -122.339494)

In [9]:
# Function to take two lat/long on two locations and calculate distance in miles
def geo_distance(coord_a, coord_b):
    
    return geodesic(coord_a, coord_b).miles

In [10]:
# Creating a column for distance in miles to Amazon HQ using the location column and amazon coordinates. 
realkcdf['distance_to_amazon'] = realkcdf['location'].apply(lambda x: geodesic(x, amzn_coord).miles)

<ipython-input-10-78d5d5f6fce4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  realkcdf['distance_to_amazon'] = realkcdf['location'].apply(lambda x: geodesic(x, amzn_coord).miles)


Using distance from Amazon HQ to filter for houses within 3 miles and remove outliers

In [11]:
# Filter data to have only the houses that were sold with in three miles of Amazon
data_near_amzn = realkcdf[realkcdf['distance_to_amazon'] <= 3]

In [12]:
# Filtering out one of the big outliers
data_near_amzn_filt = data_near_amzn[data_near_amzn['price'] < 10000000]

## Data Exploration and Visualization

In [ ]:
# Creating a pairplot to look at relationships of all numerical values
sns.pairplot(realkcdf, corner = True);

### See price correlated with size of the home and property, as well as number of bedrooms and bathrooms
#### Created a Heatmap to see specific correlation values between different features of the dataset

In [ ]:
# Plotting a heatmap to look at the collinearity of dependent variables
plt.figure(figsize = (30,15))
sns.heatmap(realkcdf.corr(), annot = True);

### Heat map correlation displays narrowed our focus to square foot of living and number of bathrooms, showing 0.62 and 0.49 correlation respectively

In [ ]:
# Plotting the relationship of the sqft of living vs the sale price
fig, ax = plt.subplots(figsize = (15,10))

x = data_near_amzn_filt['sqft_living']
y = data_near_amzn_filt['price']
hue = data_near_amzn_filt['bedrooms']
size = data_near_amzn_filt['bathrooms']


sns.scatterplot(x = x, y = y, ax=ax)
ax.set_title('Square Feet of Living vs Price (3 Mile Range from Amazon HQ)', fontsize = 22)
ax.set_ylabel('Price in Millions USD', fontsize = 16)
ax.set_xlabel('Square Feet of Living', fontsize = 16);

### We observed a strong positive correlation  between the  house size and price

In [ ]:
# Plotting the number of bathrooms and their average price
fig, ax = plt.subplots(figsize = (15,10))

x = data_near_amzn_filt['bathrooms']
y = data_near_amzn_filt['price']

sns.barplot(x, y, data = realkcdf, ax = ax)
ax.set_title('Average Price vs Bathrooms (3 Mile Range From Amazon HQ)', fontsize = 22)
ax.set_ylabel('Price in Millions USD', fontsize = 16)
ax.set_xlabel('Number of Bathrooms', fontsize = 16);

### See a linear correlation between from average price and number of bathrooms

## Data Modeling

### Created a Multivariate Linear Regression model using numerical and non-numeric columns as features

### Feature Engineering

First dealt with Oirdinal Categorical columns

In [ ]:
# Creating lists to put in the ordinal encoder
grade_list = ['2 Substandard','3 Poor','4 Low','5 Fair','6 Low Average', '7 Average', '8 Good', '9 Better', '10 Very Good', '11 Excellent','12 Luxury','13 Mansion']
cond_list = ['Poor', 'Fair', 'Average', 'Good', 'Very Good']
view_list = ['NONE', 'FAIR', 'AVERAGE', 'GOOD', 'EXCELLENT']

In [ ]:
# Ordinal encoding lists and creating an ordinal datafram
o_enc = OrdinalEncoder(categories = [grade_list, cond_list, view_list])
ord_df =pd.DataFrame(o_enc.fit_transform(realkcdf[['grade', 'condition', 'view']]), columns = realkcdf[['grade', 'condition', 'view']].columns)

In [ ]:
# Concatenating the realkcdf and ordinal dataframes and 
new_kcdf = pd.concat([realkcdf.reset_index().drop(columns = ['grade', 'condition', 'view']), ord_df],1)

Wrangled Nominal Categorical columns to use in regression model

In [ ]:
# Instantiate a Onehot Encoding object, apply to nominal categoricals and concatenate to main dataframe 
onehot_enc = OneHotEncoder(sparse = True, handle_unknown = 'ignore')
nominal_cols = ['waterfront', 'greenbelt', 'nuisance','heat_source', 'sewer_system','zip_code']
ohe_df = onehot_enc.fit_transform(new_kcdf[nominal_cols])
nominal_df = pd.DataFrame(ohe_df.toarray(),columns = onehot_enc.get_feature_names())
cleandf = pd.concat([new_kcdf.drop(columns = 
                ['waterfront', 'greenbelt', 'heat_source', 'sewer_system', 'zip_code', 'nuisance']), nominal_df],1)

### Model Creation

### First Simple Model (FSM)

In [ ]:
# Instantiating the Linear Regression object
lr = LinearRegression()

In [ ]:
# Selecting columns and using as features to train model for price
col_select1 = realkcdf.drop(columns = ['price', 'id', 'waterfront', 'nuisance', 'waterfront', 'greenbelt', 'view', 'condition', 'grade','heat_source', 'sewer_system', 'address', 'zip_code', 'location', 'distance_to_amazon' ]).columns
X = realkcdf[col_select1]
y = realkcdf['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# Fitting the data to the regression
lr.fit(X_train, y_train)

In [ ]:
# Checking R^2 score of the training data to see variation
lr.score(X_train, y_train)

In [ ]:
# Checking prediction of sale price on separated test data
y_pred = lr.predict(X_test)

In [ ]:
# Calculating the mean absolute error
MAE = mean_absolute_error(y_pred, y_test)
MAE

Observed that the coeffecient of determination is not very high and that there is a large mean absolute error

Use Standard Scaler to increase the model's efficacy

In [ ]:
# Instantiating StandardScalar and fit to training data
ss = StandardScaler()
ss.fit(X_train)

In [ ]:
# Transforming and fitting the X_train value with standard scalar
X_standardized_train = ss.fit_transform(X_train)

In [ ]:
# Using standard scalar to standardize the test set
X_standardized_test = ss.transform(X_test)

In [ ]:
# Fitting the model against standardized training data and checking the R^2 value
lr.fit(X_standardized_train, y_train)
lr.score(X_standardized_train, y_train)

In [ ]:
# The R^2 value of the test standardized and the test data test sale prices
lr.score(X_standardized_test, y_test)

In [ ]:
# Predicting the sale price of houses from the standardized test set
y_predstand = lr.predict(X_standardized_test)

In [ ]:
# Calculating the standardized Mean Absolute Error
stand_MAE = mean_absolute_error(y_predstand, y_test)
stand_MAE

#### Analysis of First  Simple Model
- In our first model we predicted price based on the simple numerical data given to us without any feature engineering.
- The R^2 value and MAE are not that great showing that our model is not a great predictor for price.
- Next step is to put in ordinal categories

### Second Model

In [ ]:
# Sustantiating the LinearRegression
lr2 = LinearRegression()

In [ ]:
# Using the ordinal columns and adding them to the model
col_select2 = new_kcdf.drop(columns = ['price', 'id', 'waterfront', 'nuisance', 'waterfront', 'greenbelt', 'heat_source', 'sewer_system', 'address', 'zip_code', 'location', 'distance_to_amazon']).columns
X = new_kcdf[col_select2]
y = new_kcdf['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# Fitting the data to the regression
lr2.fit(X_train, y_train)

In [ ]:
# R^2 score of the training data
lr2.score(X_train, y_train)

In [ ]:
# Predicted sales price from the test set
y_pred = lr2.predict(X_test)

In [ ]:
lr2.score(X_train,y_train)

In [ ]:
MAE = mean_absolute_error(y_pred, y_test)
MAE

In [ ]:
# Assigning Standard Scalar
ss = StandardScaler()

In [ ]:
# Standard scaling the X train data
ss.fit(X_train)

In [ ]:
# Transforming and fitting the X_train value with standard scalar
X_standardized_train = ss.fit_transform(X_train)

In [ ]:
# Using standard scalar to standardize the test set
X_standardized_test = ss.transform(X_test)

In [ ]:
# The R^2 value of the standardized train set and predicted price
lr2.fit(X_standardized_train, y_train)
lr2.score(X_standardized_train, y_train)

In [ ]:
# The R^2 value of the test standardized x values and the test price values
lr2.score(X_standardized_test, y_test)

In [ ]:
# Predicted price
y_predstand = lr2.predict(X_standardized_test)

In [ ]:
# Mean absolute error
stand_MAE = mean_absolute_error(y_predstand, y_test)
stand_MAE

#### Analysis of Second Model
- In our second model we incorporated the ordinal categories into our model
- The R^2 value increased and our MAE decreased which means our model prediction was better than our first simple model

### Final Model

In [ ]:
# Assigning the LinearRegression
lr3 = LinearRegression()

In [ ]:
col_select = cleandf.drop(columns = ['price', 'id', 'index', 'address','sqft_garage', 'location']).columns
X = cleandf[col_select]
y = cleandf['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# Fitting the data to the regression
lr3.fit(X_train, y_train)

In [ ]:
# R^2 score of the training data
lr3.score(X_train, y_train)

In [ ]:
#Looking at the first ten predicted sales prices from the test set
y_pred = lr3.predict(X_test)
y_pred[0:10]

In [ ]:
# The R^2 value of the test data
lr3.score(X_train,y_train)
lr3.score(X_test,y_test)

In [ ]:
# The mean absolute error of the 
MAE = mean_absolute_error(y_pred, y_test)
MAE

In [ ]:
# Assigning StandardScalar
ss = StandardScaler()

In [ ]:
# Standard scaling the X train data
ss.fit(X_train)

In [ ]:
# Transforming and fitting the X_train value with standard scalar
X_standardized_train = ss.fit_transform(X_train)

In [ ]:
# Transforming the X_test set with standard Scalar
X_standardized_test = ss.transform(X_test)

In [ ]:
# R^2 value of the standardized train set with train set sale prices
lr3.fit(X_standardized_train, y_train)
lr3.score(X_standardized_train, y_train)


In [ ]:
# The R^2 value of the test standardized x values and the test price values
lr3.score(X_standardized_test, y_test)

In [ ]:
# Predicting the prices of the standarized feature matrix and assigning it to ss_preds 
ss_preds3 = lr3.predict(X_standardized_test)

In [ ]:
# Taking a look at the predicted prices of the standardized feature matrix
ss_preds3

In [ ]:
# Predicting the sale price from the test set using the standard scalar feature matrix
y_predstand3 = lr3.predict(X_standardized_test)

In [ ]:
stand_MAE = mean_absolute_error(y_predstand3, y_test)

In [ ]:
# Calculating mean absolute error
stand_MAE

In [ ]:
# Using the ols modeling so its easier to see what coefficent weights are related to what independent variable
x = cleandf.drop(columns = ['price', 'id','index', 'address', 'sqft_garage', 'lat', 'location'])
y = cleandf['price']
results = sm.OLS(y, sm.add_constant(x)).fit()

### Model Results

In [ ]:
# Summary statistics of OLS
results.summary()

In [ ]:
# Predicted vs actual on the test
sns.regplot(x = ss_preds3, y = y_test);

#### Analysis of reggresion plot: 
- Plotted our model's predicted price vs actual price
- Can see variance of prediction accuracy at different price ranges
- See cheaper houses have a better prediction then very expensive houses

In [ ]:
# Distribution of residuals of the ss_preds minus the y_test
sns.displot(ss_preds3 - y_test);

#### Analysis of distribution plot: 
- See distribution of the residuals from our model predicted price vs actual price
- Residuals are normally distributed around 0, showing our model is an accurate predictor
- See more negative values, showing actual sale price was great than predicted price
    - This could be from individuals overpaying for a desirable house


## Conclusions
- Our final model includes all numerical, ordinal and nominal categories from our data frame
- Including these columns enhance the accuracy of our model
- The R^2 value is much higher than the first two models and is a respectable value. About 65% of the variability observed in the sale price is explained by the regression model.
- The Mean Absolute Error is also much smaller than our first model 
- Our final model is a pretty decent predictor of house prices based on certain features in King County
- Summary table also shows the influence each category to our predicted price.
- Some coefficients that increase house sale price include: bathrooms, sqft_living, sqft_above, sqft_basement, sqft_patio, grade, condition and view
- As we saw from our heatmap, sqft_living and bathrooms were most significant factors

## Recommendations
- Based on our model, we can look at the coefficents of the regression model to make recommendations to new home buyers.
- Two realistic options to look at are the coefficents of bathrooms and sqft_living
- If new home buyers want to increase the value/price of their home then we recommend that they build a new bathroom or build an extension to the house.